# Transformer - Gerador e Resumo


### Descriçao

---
1. O objetivo deste notebook é aprender sobre Transformer
2. Vamos aprender sobre a biblioteca Transformer e construir um gerador de texto e também um resumo de paragrafos.
---

### Baixando os pacotes


In [2]:
!pip install transformers

### Importando as bibliotecas

In [3]:
import transformers
from transformers import pipeline

### Gerador de Texto

In [7]:
gerador = pipeline("text-generation", model="pierreguillou/gpt2-small-portuguese")
texto = "Em sentido estrito, ciência refere-se ao sistema de adquirir conhecimento baseado no método científico."
resultado = gerador(texto, max_length=60, do_sample=True, temperature=0.7, top_k=50, top_p=0.85)
resultado[0]['generated_text']

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'Em sentido estrito, ciência refere-se ao sistema de adquirir conhecimento baseado no método científico.\n\nO conhecimento científico se desenvolve na forma de conhecimentos gerais, que são as fontes de conhecimento que são relevantes para a ciência, e que são fundamentais para a compreensão da natureza e dos fenômenos naturais.\n\n'

### Preencher Lacunas

In [8]:
mascarar = pipeline("fill-mask", model="neuralmind/bert-base-portuguese-cased")

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
texto = mascarar("Existe uma chance do copo cair no [MASK]")
for x in range(len(texto)):
  print(texto[x])

{'score': 0.5944011211395264, 'token': 8105, 'token_str': 'chão', 'sequence': 'Existe uma chance do copo cair no chão'}
{'score': 0.04544369876384735, 'token': 2187, 'token_str': 'rio', 'sequence': 'Existe uma chance do copo cair no rio'}
{'score': 0.04219300299882889, 'token': 528, 'token_str': 'mar', 'sequence': 'Existe uma chance do copo cair no mar'}
{'score': 0.03341993689537048, 'token': 4848, 'token_str': 'fogo', 'sequence': 'Existe uma chance do copo cair no fogo'}
{'score': 0.02256273850798607, 'token': 14575, 'token_str': 'lixo', 'sequence': 'Existe uma chance do copo cair no lixo'}


### Perguntas e Respostas

In [10]:
qea = pipeline("question-answering", model="pierreguillou/bert-base-cased-squad-v1.1-portuguese")

config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [11]:
texto = "Carl Sagan foi um cientista norte-americano. Sagan é autor de mais de 600 publicações científicas e também de mais de vinte livros de ciência e ficção científica."
pergunta = "Quantas publicações cientificas Carl Sagan tem publicado?"
resposta = qea(question=pergunta, context=texto)
print("Pergunta: ", pergunta)
print("Resposta: ", resposta['answer'])
print("Score: ", resposta['score'])

Pergunta:  Quantas publicações cientificas Carl Sagan tem publicado?
Resposta:  mais de 600
Score:  0.7415890693664551


### Resumo

In [13]:
resumidor = pipeline("summarization")
texto = """Tolkien criou um mundo, a Terra Média. Desenhou mapas que indicam a localização de cada rio, mata ou acidente geográfico importante. Estudioso e especialista do inglês antigo e medieval, davas aulas de lingüística e aproveitou seu conhecimento para incorporar uma série de elementos inéditos em seus livros. Assim criou um alfabeto e uma língua com regras de gramática e pronúncia. Em seguida imaginou um povo para habitar este mundo e deu a ele uma história, heróis para reverenciar e inimigos para temer. Criou um calendário, leis e uma mitologia. Reproduziu as árvores genealógicas de seus personagens principais com atenção aos detalhes de até seis gerações diferentes. Várias raças habitam esta região, incluindo seres humanos, Elfos, Trols, Duendes e, os mais famosos, os Hobbits. Todas estas informações estão em grande parte incorporadas no final do terceiro volume da trilogia, em seis apêndices somando quase 200 páginas."""
resumo = resumidor(texto, max_length=100, min_length=50)
resumo = resumo[0]['summary_text']
print(f"Resumo: {resumo}")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Resumo:  Tolkien criou um mundo, a Terra Média . Desenhou mapas that indicam a localização of cada rio, mata ou acidente geográfico importante . Criou alfabeto e língua com regras de gramática e pronúncia . Várias raças habitam esta região, incluindo seres human
